In [8]:
from utils import CONSTANTS
import pickle


class Grid():

    data_path : str = f'{CONSTANTS.AUGER_FD_ROOT}/xy_measurements.pkl'
    n_bays : dict = {t : 6 if t != 'HE' else 3 for t in['LL', 'LM', 'LA', 'CO', 'HE']}

    def __init__(self, *args, compare='none') -> None :

        telescopes = [eye.upper() for eye in args]
        
        with open(f'{CONSTANTS.AUGER_FD_ROOT}/xy_measurements.pkl', 'rb') as f:
            measurements = pickle.load(f)

        self.grid_data = {t: None for t in telescopes}
        for site in telescopes:
            n_cols = Grid.n_bays[site]
            
            for tel in [f'{site}{i}' for i in range(1, n_cols + 1)]:
                print(tel)



In [10]:
_ = Grid('ll', 'he')

LL1
LL2
LL3
LL4
LL5
LL6
HE1
HE2
HE3


In [11]:
with open(f'{CONSTANTS.AUGER_FD_ROOT}/xy_measurements.pkl', 'rb') as f:
           measurements = pickle.load(f)


In [13]:
print(len(measurements['CO5']))

2
